In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size  = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [5]:
graph = tf.Graph()
batch_size = 128
beta = 0.01

with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size,image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32,shape=(batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(tf.truncated_normal([image_size*image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=tf_train_labels)) + \
      beta*tf.nn.l2_loss(weights)
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset+batch_size)]
        batch_labels = train_labels[offset:(offset+batch_size)]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction],feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Loss at step %d: %f" % (step, l))
            print("Training accuracy: %.1f%%" % accuracy(predictions,batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 44.068317
Training accuracy: 21.1%
Validation accuracy: 18.1%
Loss at step 500: 1.118646
Training accuracy: 72.7%
Validation accuracy: 80.9%
Loss at step 1000: 0.681309
Training accuracy: 85.2%
Validation accuracy: 81.0%
Loss at step 1500: 0.750395
Training accuracy: 82.8%
Validation accuracy: 80.3%
Loss at step 2000: 0.808888
Training accuracy: 79.7%
Validation accuracy: 80.2%
Loss at step 2500: 0.721846
Training accuracy: 83.6%
Validation accuracy: 81.3%
Loss at step 3000: 0.716108
Training accuracy: 79.7%
Validation accuracy: 80.9%
Test accuracy: 88.5%


In [7]:
graph = tf.Graph()
batch_size = 128
num_hidden = 1024
beta = 0.01

with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size,image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32,shape=(batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights_1 = tf.Variable(tf.truncated_normal([image_size*image_size, num_hidden]))
    biases_1 = tf.Variable(tf.zeros([num_hidden]))
    
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    prob_1 = tf.nn.relu(logits_1)
    
    weights_2 = tf.Variable(tf.truncated_normal([num_hidden, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    logits_2 = tf.matmul(prob_1, weights_2) + biases_2
    loss = tf.reduce_mean(tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits_2,labels=tf_train_labels)) + \
      beta*(tf.nn.l2_loss(weights_1)+tf.nn.l2_loss(weights_2)))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits_2)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) +\
      biases_2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) +\
      biases_2)

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset+batch_size)]
        batch_labels = train_labels[offset:(offset+batch_size)]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction],feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Loss at step %d: %f" % (step, l))
            print("Training accuracy: %.1f%%" % accuracy(predictions,batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 3547.283203
Training accuracy: 9.4%
Validation accuracy: 23.6%
Loss at step 500: 21.451242
Training accuracy: 78.1%
Validation accuracy: 83.8%
Loss at step 1000: 0.846743
Training accuracy: 84.4%
Validation accuracy: 82.9%
Loss at step 1500: 0.693049
Training accuracy: 85.2%
Validation accuracy: 83.4%
Loss at step 2000: 0.769948
Training accuracy: 81.2%
Validation accuracy: 83.5%
Loss at step 2500: 0.715051
Training accuracy: 82.0%
Validation accuracy: 83.5%
Loss at step 3000: 0.666773
Training accuracy: 84.4%
Validation accuracy: 84.0%
Test accuracy: 90.8%


In [10]:
graph = tf.Graph()
batch_size = 128
num_hidden = 1024
beta = 0.01

with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size,image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32,shape=(batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights_1 = tf.Variable(tf.truncated_normal([image_size*image_size, num_hidden]))
    biases_1 = tf.Variable(tf.zeros([num_hidden]))
    
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    prob_1 = tf.nn.relu(logits_1)
    
    weights_2 = tf.Variable(tf.truncated_normal([num_hidden, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    logits_2 = tf.matmul(prob_1, weights_2) + biases_2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits_2,labels=tf_train_labels))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits_2)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) +\
      biases_2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) +\
      biases_2)

In [11]:
num_steps = 3001
train_dataset_2 = train_dataset[:1501]
train_labels_2 = train_labels[:1501]

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels_2.shape[0] - batch_size)
        batch_data = train_dataset_2[offset:(offset+batch_size)]
        batch_labels = train_labels_2[offset:(offset+batch_size)]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction],feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Loss at step %d: %f" % (step, l))
            print("Training accuracy: %.1f%%" % accuracy(predictions,batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 520.745667
Training accuracy: 6.2%
Validation accuracy: 35.6%
Loss at step 500: 0.000001
Training accuracy: 100.0%
Validation accuracy: 77.9%
Loss at step 1000: 0.613188
Training accuracy: 99.2%
Validation accuracy: 77.5%
Loss at step 1500: 0.000149
Training accuracy: 100.0%
Validation accuracy: 77.7%
Loss at step 2000: 0.000000
Training accuracy: 100.0%
Validation accuracy: 77.4%
Loss at step 2500: 0.000000
Training accuracy: 100.0%
Validation accuracy: 77.6%
Loss at step 3000: 0.503703
Training accuracy: 99.2%
Validation accuracy: 77.1%
Test accuracy: 85.3%


In [12]:
graph = tf.Graph()
image_size=28
num_labels=10
batch_size = 128
num_hidden = 1024
beta = 0.01

with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size,image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32,shape=(batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights_1 = tf.Variable(tf.truncated_normal([image_size*image_size, num_hidden]))
    biases_1 = tf.Variable(tf.zeros([num_hidden]))
    
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    prob_1 = tf.nn.dropout(tf.nn.relu(logits_1),tf.constant(0.5))
    
    weights_2 = tf.Variable(tf.truncated_normal([num_hidden, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    logits_2 = tf.matmul(prob_1, weights_2) + biases_2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits_2,labels=tf_train_labels)) 
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(logits_1), weights_2) + biases_2)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) +\
      biases_2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) +\
      biases_2)

In [13]:
num_steps = 3001
train_dataset_2 = train_dataset[:1501]
train_labels_2 = train_labels[:1501]

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels_2.shape[0] - batch_size)
        batch_data = train_dataset_2[offset:(offset+batch_size)]
        batch_labels = train_labels_2[offset:(offset+batch_size)]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction],feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Loss at step %d: %f" % (step, l))
            print("Training accuracy: %.1f%%" % accuracy(predictions,batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 484.148346
Training accuracy: 6.2%
Validation accuracy: 30.8%
Loss at step 500: 2.443954
Training accuracy: 100.0%
Validation accuracy: 79.8%
Loss at step 1000: 10.267498
Training accuracy: 97.7%
Validation accuracy: 80.9%
Loss at step 1500: 0.373242
Training accuracy: 100.0%
Validation accuracy: 80.8%
Loss at step 2000: 0.000000
Training accuracy: 100.0%
Validation accuracy: 81.0%
Loss at step 2500: 0.004257
Training accuracy: 100.0%
Validation accuracy: 80.9%
Loss at step 3000: 0.000000
Training accuracy: 99.2%
Validation accuracy: 80.9%
Test accuracy: 88.4%
